In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [13]:
from dotenv import load_dotenv
load_dotenv()
import os

In [14]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [15]:
#Arxiv Tool for research papers
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")


In [16]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [17]:
#Wikipedia tool for wikipedia articles
api_wrapper_wikipedia=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia,description="Query Wikipedia articles")

In [18]:
#Tavily tool for web-search
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults(api_key=TAVILY_API_KEY,description="Query web-search results")
tavily.invoke("tell me recent US tariff news")

[{'url': 'https://apnews.com/article/trump-tariffs-liberation-day-2a031b3c16120a5672a6ddd01da09933',
  'content': 'WASHINGTON (AP) — President Donald Trump announced far-reaching new tariffs on nearly all U.S. trading partners — a 34% tax on imports from China and 20% on the European Union, among others — that threaten to dismantle much of the architecture of the global economy and trigger broader trade wars. Just days after Trump unveiled sweeping new “reciprocal” tariffs on imports from around the world, China levied a retaliatory 34% tariff on all U.S. products starting April 10.AP News Just days after Trump unveiled sweeping new “reciprocal” tariffs on imports from around the world, China levied a retaliatory 34% tariff on all U.S. products starting April 10.AP News'},
 {'url': 'https://finance.yahoo.com/news/economists-continue-to-revise-estimates-for-trumps-tariff-policy-most-possibilities-still-take-us-back-to-the-1910s-184137558.html',
  'content': '"The upshot is that the over

In [20]:
#Putting tools in one list
tools=[arxiv,wikipedia,tavily]

In [26]:
#Introducing LLM
from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [27]:
llm.invoke("Hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.036363636, 'prompt_time': 0.001912311, 'queue_time': 0.055617269, 'total_time': 0.038275947}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run-eee70eca-e1f4-44b3-bf22-61f9ed4d7ebb-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [29]:
llm_with_tools=llm.bind_tools(tools)

In [30]:
llm_with_tools.invoke("Tell me recent research on AI")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vpbg', 'function': {'arguments': '{"query": "recent AI research"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 339, 'total_tokens': 356, 'completion_time': 0.067945223, 'prompt_time': 0.028549974, 'queue_time': 0.057100959, 'total_time': 0.096495197}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-faaef622-2e7b-48bb-8500-ed90b511e389-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'recent AI research'}, 'id': 'call_vpbg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 339, 'output_tokens': 17, 'total_tokens': 356})

## Workflow

In [62]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage #Human msg or AI msg
from typing import Annotated
from langgraph.graph.message import add_messages #Reducers in Langgraph

In [36]:
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [49]:
from IPython.display import Image, display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [50]:
#Node definition
def assistant(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}


In [70]:
builder=StateGraph(State)
builder.add_node("Assistant",assistant)
builder.add_node("tools",ToolNode(tools))
builder.add_edge(START,"Assistant")
builder.add_conditional_edges("Assistant",tools_condition) # Conditional tool call, if tool needed edge to tool else edge to END
builder.add_edge("tools","Assistant") #Tool call edge to assistant
builder.add_edge("Assistant",END)

In [71]:
graph=builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [65]:
messages=graph.invoke({"messages":"1706.03762"})
for i in messages['messages']:
    i.pretty_print()

================================ Human Message =================================

1706.03762
================================== Ai Message ==================================
Tool Calls:
  arxiv (call_ct62)
 Call ID: call_ct62
  Args:
    query: 1706.03762
================================= Tool Message =================================
Name: arxiv

Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, base


In [73]:
messages=graph.invoke({"messages":"Hello, what's up please tell me about the live scorre of today's IPL match"})
for i in messages['messages']:
    i.pretty_print()

================================ Human Message =================================

Hello, what's up please tell me about the live scorre of today's IPL match
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_0jvv)
 Call ID: call_0jvv
  Args:
    query: IPL match live score today
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.hindustantimes.com/cricket/live-score", "content": "Tune in to Crickit for the live IPL score, ball-by-ball updates and cricket today in Match 31 of IPL 2025.\n\nTune in to Crickit for all the live cricket score from IPL 2025 and cricket today matches from the Indian Premier League.\n\nStay tuned to this page for IPL live scores, ball-by-ball commentary, stats and match highlights. Whether you are watching from home or following on the go, we have got every over of IPL 2025 covered for you. [...] IPL,